In [1]:
import time
from collections import deque, namedtuple
import snake_env
import gymnasium
import numpy as np
import tensorflow as tf


c:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [227]:
MEMORY_SIZE = 100_000     # size of memory buffer
GAMMA = 0.995             # discount factor
ALPHA = 1e-3              # learning rate  
NUM_STEPS_FOR_UPDATE = 4  # perform a learning update every C time steps

In addition, we also have to initialise our hyper parameters
MEMORY_SIZE determines how large the buffer is, used to store experiences
GAMMA is a discount where the future reward have a tendency to be decreased overtime
ALPHA is the learning rate we used for the update in deep Q learning
NUM_STEPS_FOR_UPDATE determines how many steps we have to acquire before updating the q network and soft updating the target q network

import all necessary library to build ourself deep q learning 

In [2]:
env = snake_env.SnakeEnv()

We set up some display so the user can see inside the box

In [156]:
env.reset()

(array([0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1.], dtype=float32),
 {'score': 0})

but first, we have to reset our environment to the initial state

In [44]:
state_size = env.observation_space.shape
num_actions = env.action_space.n
print('State Shape:', state_size)
print('Number of actions:', num_actions)

State Shape: (11,)
Number of actions: 4


In [221]:
observation, reward, terminated, truncated, _ = env.step(action = 3)
print(observation)
print(reward)
print(terminated)

[0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1.]
0
False


    ACTION:
    -UP = 0
    -RIGHT = 1
    -DOWN = 2
    -LEFT = 3
    OBSERVATION
    -danger_straight
    -danger_right
    -danger_left
    -dir_up
    -dir_right
    -dir_down
    -dir_left,
    -food_up
    -food_down
    -food_left
    -food_right


In [226]:
q_network = tf.keras.Sequential([
    tf.keras.Input(shape = state_size),
    tf.keras.layers.Dense(units = 64, activation = 'relu'),
    tf.keras.layers.Dense(units = 64, activation = 'relu'),
    tf.keras.layers.Dense(units = num_actions, activation = 'linear')
    ])
target_q_network = tf.keras.Sequential([
    tf.keras.Input(shape = state_size),
    tf.keras.layers.Dense(units = 64, activation = 'relu'),
    tf.keras.layers.Dense(units = 64, activation = 'relu'),
    tf.keras.layers.Dense(units = num_actions, activation = 'linear')
    ])

Next we will initialize 2 networks, q network and target q network, both share the same architecture. We can only just initialize 1 network only, however, for alot of reasons: such as for stable learning, avoiding oscillations, etc..
We will use 2 networks which will help our agent stable the learning

In [228]:
optimizer = tf.keras.optimizers.Adam(learning_rate= ALPHA)

Initialize our Adam optimizer